In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

# One tailed t-test - 
In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/machine.txt. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

In [12]:
#filename = 'C:/Users/mahna/Ironhack/Week7/Day2/lab-t-tests-p-values/files_for_lab/machine.txt'

# machine_data = pd.read_csv('C:/Users/mahna/Ironhack/Week7/Day2/lab-t-tests-p-values/files_for_lab/machine.txt', encoding= 'utf-16', sep='\t')

In [2]:
df = pd.read_csv("files_for_lab/machine.txt", encoding = "utf-16", sep = "\t", names = ["new", "old"], header=0)

In [3]:
df

,new,old
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


Let's state our Hypothesis

$$H0: \mu_{New machine} < \mu_{Old machine} $$
$$H1: \mu_{New machine} => \mu_{Old machine} $$

Given the inequality on the alternative, we need to use a "right-sided" test.

In [4]:
old_mean, new_mean = df['old'].mean(), df['new'].mean()
old_std, new_std = df['old'].std(ddof=1), df['new'].std(ddof=1)

In [5]:
sample_old, sample_new = df['old'], df['new']

In [6]:
sp = ( len(sample_old) - 1 ) * ( old_std**2 ) +  ( len(sample_new) - 1 ) * ( new_std**2 )
sp /= ( len(sample_old) + len(sample_new) - 2)
sp = np.sqrt(sp)
r = np.sqrt( (1/len(sample_old)) + (1/len(sample_new)) )
t = ( old_mean - new_mean )/ (sp * r)

print("The mean of old machine is {:.2f}".format(old_mean))
print("The mean of new machine is {:.2f}".format(new_mean))
print("The standard deviation of old machine is {:.2f}".format(old_std))
print("The standard deviation of new machine is {:.2f}".format(new_std))
print("The ratio of the old and new variances is {:.2f} which is bigger than 0.5 and smaller than 2".format(old_std/new_std))
print("The t statistic is: {:.2f}".format(t))

The mean of old machine is 43.23
The mean of new machine is 42.14
The standard deviation of old machine is 0.75
The standard deviation of new machine is 0.68
The ratio of the old and new variances is 1.10 which is bigger than 0.5 and smaller than 2
The t statistic is: 3.40


Now let's do the test. We need to remember that this is a one/right-tailed test.

Let's do the test first with critical values:

In [7]:
st.t.ppf(1-(0.05), df = len(sample_old)+len(sample_new)-2)

1.7340636066175354

The t statistic lies on the right of the critical value. So, H0 rejected. Therefore, new machine takes equal or more time than the old machine.

with p_value:

In [8]:
st.t.cdf(t,df = len(sample_old)+len(sample_new)-2)

0.9983944287496127

this value is bigger than 0.95, so H0 rejected!

# Matched Pairs Test - 
In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment your result.

In [9]:
data = pd.read_csv('C:/Users/mahna/Ironhack/Week7/Day2/lab-t-tests-p-values/files_for_lab/pokemon.csv')
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


$$H0: \mu_{Defense} = \mu_{Attack} → \mu_{Defense} - \mu_{Attack} = 0$$
$$H1: \mu_{Defense} \neq \mu_{Attack} → \mu_{Defense} - \mu_{Attack} \neq 0$$

Given the unequality on the alternative, we need to use a "two-sided" test.

In [10]:
sample = data.sample(30, random_state = 1)
sample['difference'] = sample['Defense']-sample['Attack']
sample.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,difference
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False,-26
510,460,Abomasnow,Grass,Ice,494,90,92,75,92,85,60,4,False,-17
175,161,Sentret,Normal,NaN,215,35,46,34,35,45,20,2,False,-12
735,667,Litleo,Fire,Normal,369,62,50,58,73,54,72,6,False,8
242,224,Octillery,Water,NaN,480,75,105,75,105,75,45,2,False,-30


In [11]:
sample_diff_mean, sample_diff_std = sample['difference'].mean(), sample['difference'].std(ddof=1)
sample_diff_mean, sample_diff_std

(-2.8, 36.32079294288603)

Out t statistic is:

$$t = \frac{\overline{d}}{\frac{\sigma}{\sqrt{n}}}$$

where $\overline{d} = mean(sample_defense - sample_attack)$

In [12]:
sample.shape[0]

30

In [13]:
t = sample_diff_mean / ( sample_diff_std / np.sqrt(sample.shape[0]) )
print("The mean of our samples differences is: {:.2f}".format(sample_diff_mean))
print("The standard deviation of our samples differences is: {:.2f}".format(sample_diff_std))
print("Our t statistics is: {:.2f}".format(t))

The mean of our samples differences is: -2.80
The standard deviation of our samples differences is: 36.32
Our t statistics is: -0.42


In [14]:
tc = st.t.ppf(1-(0.05/2),df= sample.shape[0] - 1)
tc

2.045229642132703

In [15]:
tcl = st.t.ppf((0.05/2),df= sample.shape[0] - 1)
tcl

-2.0452296421327034

The t statistic lies within tc and tcl. So we accept H0.

### with P value

In [16]:
1-st.t.cdf(t,df = sample.shape[0] - 1)

0.6620194128841399

This value is with 0.025 and 0.975. Therefore, we accept the null hypothesis.

## Part 1
In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on: - Null hypothesis - Alternate hypothesis - Level of significance - Test statistic - P-value - F table

Context
Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder

State the null hypothesis
State the alternate hypothesis
What is the significance level
What are the degrees of freedom of model, error terms, and total DoF

$$H0: \mu_{1}=\mu_{2}=\mu_{3}=⋯=\mu_{n}$$
$$H1: \mu_{i}\neq\mu_{j}$$

for any i, j


Level of significance 0.05

In [17]:
d = pd.read_excel('C:/Users/mahna/Ironhack/Week7/Day2/lab-t-tests-p-values/files_for_lab/anova_lab_data.xlsx', names = ["power", "etching_rate"], header=0)
d

,power,etching_rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [18]:
def wless(x):
    
    x = x.replace(' W', "")
    x = x.replace('W', "")
    x = int(x)

    return x

In [19]:
# d['Power'] = d['Power'].apply(wless)
d['power'] = list(map(wless,d['power']))

In [20]:
d

,power,etching_rate
0,160,5.43
1,180,6.24
2,200,8.79
3,160,5.71
4,180,6.71
5,200,9.20
6,160,6.22
7,180,5.98
8,200,7.90
9,160,6.01


In [21]:
group_df = d.groupby('power')['etching_rate'].agg(etching_rate_mean='mean',Samples='size').reset_index()
group_df

,power,etching_rate_mean,Samples
0,160,5.792,5
1,180,6.238,5
2,200,8.318,5


In [25]:
print(st.f_oneway(d[d['power'] == 160]['etching_rate'],d[d['power'] == 180]['etching_rate'],
                  d[d['power'] == 180]['etching_rate'],d[d['power'] == 200]['etching_rate'],
                  d[d['power'] == 160]['etching_rate'],d[d['power'] == 200]['etching_rate']))

F_onewayResult(statistic=29.503163760804043, pvalue=1.6607495592622612e-09)


The p value is less than 0.05, therefore we reject null hypothesis.

In [26]:
d1 = d['power'].nunique() - 1
d2 = len(d) - d['power'].nunique()

print("Number of degrees of freedom d1: ",d1)
print("Number of degrees of freedom d2: ",d2)

Number of degrees of freedom d1:  2
Number of degrees of freedom d2:  12


The shape of the F distribution depends on two sets of degrees of freedom:  $d_{1}=K-1$ and $d_{2}=N-K$ 

the F statistic lies beyond d2. Therefore, we reject null hypothesis.